# Percepción (PER): examen de prácticas del bloque 2, grupo 3CO21, turno 2, 30-5-2023, 18:15:19:00

Lee este cuaderno y realiza las actividades y ejercicios propuestos.

## Lectura de MNIST

Ejecuta el código siguiente:

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784")
data = mnist.data
targets = mnist.target 

# Comentar estas dos líneas si da error al ya ser datos numpy
targets=targets.to_numpy()
data=data.to_numpy()

targets=np.int8(targets)
data=np.float32(data)

c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## Partición de los datos

El código siguiente particiona los datos en 3 subconjuntos; training, validación y test; con un 70%, 15% y 15% del total, respectivamente. Ejecuta el código siguiente con **random_state** igual a las tres últimas cifras de tu DNI/NIE:

In [2]:
from sklearn.model_selection import train_test_split
x_train_val, x_test, y_train_val, y_test  = train_test_split(data, targets, test_size=0.15, random_state=921)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.15/0.85, random_state=921) 
print(x_train.shape, x_val.shape, x_test.shape)

(49000, 784) (10500, 784) (10500, 784)


## PCA y KNN: hiper-parámetros

Estudia la precisión en validación del clasificador por el vecino más cercano entrenado con el 70% de datos reservados para entrenamiento. El número de componentes va a ser reducido con PCA. Una vez reducido prueba diferentes valores del hiper-parámetro **K** (k vecinos). Por brevedad, se recomienda probar los siguientes valores de PCA=[8,16,32] junto con los valores de K=[1,3,4,5].

In [10]:
# Escribe aquí tu solución:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

max_acc = 0
best_k = 0
best_nc = 0

for nc in [8,16,32]:
    for k in [1,3,4,5]:
        kn = make_pipeline(
            PCA(n_components=nc),
            KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
        ) 

        kn.fit(x_train, y_train)
        y_pred = kn.predict(x_val)
        acc = accuracy_score(y_val, y_pred)

        if acc > max_acc:
            max_acc = acc
            best_k = k
            best_nc = nc
print(f'La precisión MAX para el conjunto de entrenamiento es {max_acc:.2%} con {best_k} Vecinos, y {best_nc} Componentes')

La precisión MAX para el conjunto de entrenamiento es 97.91% con 3 Vecinos, y 32 Componentes


## PCA y KNN: precisión en test

Entrena ahora el KNN con el 85% de datos reservado para entrenamiento y validación, así como con la combinación de valores de **numero de componentes** de PCA y **K** que mayor precisión en validación haya obtenido en el estudio anterior. Obtén la precisión en test del mismo.

In [11]:
# Escribe aquí tu solución:
kn = make_pipeline(
            PCA(n_components=best_nc),
            KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1)
        ) 

kn.fit(x_train_val, y_train_val)
y_pred = kn.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(f'La precisión de {kn} para el conjunto de train_val / test es {acc:.2%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=3))]) para el conjunto de train_val / test es 97.48%


In [12]:
print(best_nc)

32


In [13]:
print(best_k)

3
